In [99]:
import nltk
import re

In [100]:
s1 = "Holmes sat."
s2 = "Holmes lit a pipe."
s3 = "We arrived the day before Thursday."
s4 = "Holmes sat in the red armchair and he chuckled."
s5 = "My companion smiled an enigmatical smile. "
s6 = "Holmes chuckled to himself."
s7 = "She never said a word until we were at the door here."
# s7a = "She never said a word."
# s7b = "We were at the door here."
s8 = "Holmes sat down and lit his pipe."
s9 = "I had a country walk on Thursday and came home in a dreadful mess."
# s9a = "I had a country walk on Thursday."
# s9b = "I came home in a dreadful mess."
s10 = "I had a little moist red paint in the palm of my hand."
# s10a = "I had a little moist red paint."
# s10b = "Holmes sat in the palm of my hand."
s11 = "Holmes sat in the little red armchair."
s12 = "Holmes sat in the the armchair."
# sentences = [s1, s2, s3, s4, s5, s6, s7, s7a, s7b, s8, s9, s9a, s9b, s10, s10a, s10b, s11, s12]
sentences = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12]
sentence = s7

In [101]:
def preprocess(sentence):
    """
    Convert `sentence` to a list of its words.
    Pre-process sentence by converting all characters to lowercase
    and removing any word that does not contain at least one alphabetic
    character.
    """
    tokens = nltk.word_tokenize(sentence)
    s = [w.lower() for w in tokens if re.search('[a-zA-Z]', w) is not None]
    return s

In [102]:
TERMINALS = """
Adj -> "country" | "dreadful" | "enigmatical" | "little" | "moist" | "red"
Adv -> "down" | "here" | "never"
Conj -> "and" | "until"
Det -> "a" | "an" | "his" | "my" | "the"
N -> "armchair" | "companion" | "day" | "door" | "hand" | "he" | "himself"
N -> "holmes" | "home" | "i" | "mess" | "paint" | "palm" | "pipe" | "she"
N -> "smile" | "thursday" | "walk" | "we" | "word"
P -> "at" | "before" | "in" | "of" | "on" | "to"
V -> "arrived" | "came" | "chuckled" | "had" | "lit" | "said" | "sat"
V -> "smiled" | "tell" | "were"
"""

NONTERMINALS = """
S -> NP VP | S Conj S
NP -> Nom | Det Nom | Nom Adv | Det Nom Adv | Det Nom PP | NP Conj NP
VP -> V | V NP | V PP | V NP PP | Adv VP | V Adv | VP Conj VP
PP -> P NP
Nom -> N | Adj Nom
"""
# NP -> Nom | Det Nom | Det Nom PP | Nom Adv |               NP Conj NP
# NP -> Nom | Det Nom | Det Nom PP | Nom Adv | Det Nom Adv | NP Conj NP

# VP -> V | V NP | V PP | V NP PP | Adv VP | V Adv | VP Conj VP
# VP -> V | V NP | V PP |           Adv VP | V Adv | VP Conj VP

# PP -> P NP
# PP -> P Nom | P Det Nom | P NP

# Nom -> N | Adj Nom
grammar = nltk.CFG.fromstring(NONTERMINALS + TERMINALS)
parser = nltk.ChartParser(grammar)

In [103]:
# marad
for sentence in sentences:
    
    s = preprocess(sentence)
    
    # Attempt to parse sentence
    try:
        trees = list(parser.parse(s))
    except ValueError as e:
        print(e)
    if not trees:
        print("Could not parse sentence.")
    
    for tree in trees:
        tree.pretty_print()
        print('\n\n')

        out = []
        for s in tree.subtrees(lambda t: t.label() == 'NP'):
            y = [x.label() for x in s.subtrees()]
            c = y.count('NP')
            if c == 1:
                out.append(s)

        print("Noun Phrase Chunks")
        for np in out:
            print(" ".join(np.flatten()))
        print('\n\n\n\n')

        S     
   _____|___   
  NP        | 
  |         |  
 Nom        VP
  |         |  
  N         V 
  |         |  
holmes     sat




Noun Phrase Chunks
holmes





            S              
   _________|___            
  |             VP         
  |      _______|___        
  NP    |           NP     
  |     |        ___|___    
 Nom    |       |      Nom 
  |     |       |       |   
  N     V      Det      N  
  |     |       |       |   
holmes lit      a      pipe




Noun Phrase Chunks
holmes
a pipe





             S                             
  ___________|_______                       
 |                   VP                    
 |      _____________|__________            
 |     |         |              PP         
 |     |         |         _____|_____      
 NP    |         NP       |           NP   
 |     |      ___|___     |           |     
Nom    |     |      Nom   |          Nom   
 |     |     |       |    |           |     
 N     V    Det      N    